In [1]:
! pip install beautifulsoup4

In [10]:
! pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.7/22.7 MB 1.2 MB/s eta 0:00:0000:0100:01m


In [2]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
def create_parser(url):
    headers = {'User-agent': 'Chrome/108.0.0.0'} 
    request = requests.get(url, headers=headers) 
    html = request.content
    parser = BeautifulSoup(html, 'html.parser') 
    
    return parser


In [36]:
init_parser = create_parser('https://www.zf.ro/burse-fonduri-mutuale/arhiva/2021/01/')

In [37]:
%%time
#21:08 start
news_df = pd.DataFrame()
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
#months = ['04', '05', '06', '07', '08', '09', '10', '11', '12']
for month in months:
    print('Working on month: {}'.format(month))
    init_parser = create_parser('https://www.zf.ro/burse-fonduri-mutuale/arhiva/2021/' + month + '/')
    max_page = 1
    for parsed in init_parser.find_all("li"):
        if 'title' in parsed.find_next('a').attrs:
            if 'pagina' in parsed.find_next('a')['title']:
                if int(parsed.find_next('a')['title'][-1:]) > max_page:
                    max_page = int(parsed.find_next('a')['title'][-1:])
    for i in range(1, max_page+1):
        print('Working on page: {}'.format(i))
        
        parser = create_parser('https://www.zf.ro/burse-fonduri-mutuale/arhiva/2021/'+ month + '/page/' + str(i))
        
        for article_h2 in parser.findAll('h2'):
            href = article_h2.find_next('a', href=True)['href']
            title = article_h2.find_next('a', href=True)\
                                          .contents[0]\
                                          .get_text()\
                                          .strip()
            try:
                parser_article = create_parser('https://www.zf.ro' + article_h2.find_next('a', href=True)['href'])

                json_scr = json.loads(parser_article.find_all('script')[2].text, strict = False)
                date_published = json_scr['datePublished']
                headline = json_scr['headline']
                article_section = json_scr['articleSection']
                description = json_scr['description']
                if 'name' in json_scr['author']:
                    author_name = json_scr['author']['name']
                else:
                    author_name = None
                article_text = json_scr['articleBody']
                rec_refs = parser_article.find_all("a", {"class": "thumb picture-type"})
                rec_hrefs = []
                for ref in range(len(rec_refs)):
                    rec_hrefs.append(rec_refs[ref]['href'])

                dict_append = {'href': href,
                              'title': title,
                              'date_published': date_published,
                              'headline': headline,
                              'article_section': article_section,
                              'description': description,
                              'author_name': author_name,
                              'article_text': article_text,
                              'rec_hrefs': rec_hrefs}
                news_df = news_df.append(dict_append, ignore_index = True)
            except:
                continue

Working on month: 01
Working on page: 1
Working on page: 2
Working on page: 3
Working on page: 4
Working on page: 5
Working on month: 02
Working on page: 1
Working on page: 2
Working on page: 3
Working on page: 4
Working on page: 5
Working on month: 03
Working on page: 1
Working on page: 2
Working on page: 3
Working on page: 4
Working on page: 5
Working on month: 04
Working on page: 1
Working on page: 2
Working on page: 3
Working on page: 4
Working on page: 5
Working on month: 05
Working on page: 1
Working on page: 2
Working on page: 3
Working on page: 4
Working on page: 5
Working on month: 06
Working on page: 1
Working on page: 2
Working on page: 3
Working on page: 4
Working on page: 5
Working on month: 07
Working on page: 1
Working on page: 2
Working on page: 3
Working on page: 4
Working on page: 5
Working on month: 08
Working on page: 1
Working on page: 2
Working on page: 3
Working on page: 4
Working on page: 5
Working on month: 09
Working on page: 1
Working on page: 2
Working on pa

In [38]:
news_df.to_parquet('datasets/df_burse_fonduri_mutuale_2021.parquet') 

In [39]:
news_df

,href,title,date_published,headline,article_section,description,author_name,article_text,rec_hrefs
0,/burse-fonduri-mutuale/actiunile-rompetrol-wel...,"Acţiunile Rompetrol Well Services, scădere de ...",2021-02-01T00:00+03:00,"Acţiunile Rompetrol Well Services, scădere de ...",Burse - Fonduri mutuale,"Acţionarii Rompetrol Well Services, divizia de...",Roxana Rosu,"\nAcţionarii Rompetrol Well Services, divizia...",[/burse-fonduri-mutuale/kmg-international-chea...
1,/burse-fonduri-mutuale/crestere-de-6-pentru-ac...,Crestere de 6% pentru acţiunile Conpet Ploieşt...,2021-02-01T00:00+03:00,Crestere de 6% pentru acţiunile Conpet Ploieşt...,Burse - Fonduri mutuale,"Acţionarii Conpet Ploieşti (COTE), transportat...",Roxana Rosu,"\nAcţionarii Conpet Ploieşti (COTE), transpor...",[/burse-fonduri-mutuale/luna-ianuarie-2021-la-...
2,/burse-fonduri-mutuale/hidroelectrica-cheama-f...,Hidroelectrica cheamă Fondul Proprietatea şi M...,2021-02-01T00:00+03:00,Hidroelectrica cheamă Fondul Proprietatea şi M...,Burse - Fonduri mutuale,"Hidroelectrica, cel mai mare producător de ene...",Liviu Popescu,"\nHidroelectrica, cel mai mare producător de ...",[/burse-fonduri-mutuale/hidroelectrica-vrea-in...
3,/burse-fonduri-mutuale/deschiderea-de-astazi-c...,DESCHIDEREA DE ASTĂZI. Cum va merge bursa. Urm...,2021-01-31T17:27:11+03:00,DESCHIDEREA DE ASTĂZI. Cum va merge bursa. Urm...,Burse - Fonduri mutuale,,Robert Manea,,[]
4,/business-hi-tech/elrond-start-up-ul-sibiu-tin...,"Elrond, start-up-ul din Sibiu care ţinteşte o ...",2021-01-31T15:00+03:00,"Elrond, start-up-ul din Sibiu care ţinteşte o ...",Business Hi-Tech,"Compania sibiană Elrond Network, care a dezvol...",Adrian Seceleanu,"\nCompania sibiană Elrond Network, care a dez...",[]
...,...,...,...,...,...,...,...,...,...
1419,/auto/swiss-capital-recomanda-strong-buy-pentr...,Swiss Capital recomandă „strong buy“ pentru ac...,2021-12-20T07:41+03:00,Swiss Capital recomandă „strong buy“ pentru ac...,Auto,"Analiştii de la Swiss Capital, unul dintre cei...",Eduard Ivanovici,\n ♦ Swiss Capital a fost broker al listă...,[/burse-fonduri-mutuale/aquila-part-prod-profi...
1420,/burse-fonduri-mutuale/zf-live-adrian-tanase-c...,"ZF Live. Adrian Tănase, CEO, BVB: Obiectivul p...",2021-12-20T07:22+03:00,"ZF Live. Adrian Tănase, CEO, BVB: Obiectivul p...",Burse - Fonduri mutuale,"Adrian Tănase, CEO al Bursei de Valori Bucureş...",Alexandra Matei,\n ♦ Energy Futures va fi un element de ...,[/burse-fonduri-mutuale/valoarea-tranzactiilor...
1421,/burse-fonduri-mutuale/facturile-mari-curent-l...,Facturile mai mari la curent şi o lună slabă p...,2021-12-20T00:07+03:00,Facturile mai mari la curent şi o lună slabă p...,Burse - Fonduri mutuale,Pe fondul creşterii peste aşteptări a inflaţie...,Liviu Popescu,\n ♦ Faţă de începutul anului 2021 acest...,[/burse-fonduri-mutuale/de-la-infiintare-fondu...
1422,/business-international/blackrock-avertizeaza-...,BlackRock avertizează că orice investiţii în o...,2021-12-20T00:00+03:00,BlackRock avertizează că orice investiţii în o...,Business Internaţional,BlackRock avertizează investitorii să nu-şi în...,Catalina Apostoiu,\nBlackRock avertizează investitorii să nu-şi...,[]


In [40]:
init_parser = create_parser('https://www.zf.ro/burse-fonduri-mutuale/arhiva/2020/01/')
#21:08 start
news_df = pd.DataFrame()
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
#months = ['04', '05', '06', '07', '08', '09', '10', '11', '12']
for month in months:
    print('Working on month: {}'.format(month))
    init_parser = create_parser('https://www.zf.ro/burse-fonduri-mutuale/arhiva/2020/' + month + '/')
    max_page = 1
    for parsed in init_parser.find_all("li"):
        if 'title' in parsed.find_next('a').attrs:
            if 'pagina' in parsed.find_next('a')['title']:
                if int(parsed.find_next('a')['title'][-1:]) > max_page:
                    max_page = int(parsed.find_next('a')['title'][-1:])
    for i in range(1, max_page+1):
        print('Working on page: {}'.format(i))
        
        parser = create_parser('https://www.zf.ro/burse-fonduri-mutuale/arhiva/2020/'+ month + '/page/' + str(i))
        
        for article_h2 in parser.findAll('h2'):
            href = article_h2.find_next('a', href=True)['href']
            title = article_h2.find_next('a', href=True)\
                                          .contents[0]\
                                          .get_text()\
                                          .strip()
            try:
                parser_article = create_parser('https://www.zf.ro' + article_h2.find_next('a', href=True)['href'])

                json_scr = json.loads(parser_article.find_all('script')[2].text, strict = False)
                date_published = json_scr['datePublished']
                headline = json_scr['headline']
                article_section = json_scr['articleSection']
                description = json_scr['description']
                if 'name' in json_scr['author']:
                    author_name = json_scr['author']['name']
                else:
                    author_name = None
                article_text = json_scr['articleBody']
                rec_refs = parser_article.find_all("a", {"class": "thumb picture-type"})
                rec_hrefs = []
                for ref in range(len(rec_refs)):
                    rec_hrefs.append(rec_refs[ref]['href'])

                dict_append = {'href': href,
                              'title': title,
                              'date_published': date_published,
                              'headline': headline,
                              'article_section': article_section,
                              'description': description,
                              'author_name': author_name,
                              'article_text': article_text,
                              'rec_hrefs': rec_hrefs}
                news_df = news_df.append(dict_append, ignore_index = True)
            except:
                continue
news_df.to_parquet('datasets/df_burse_fonduri_mutuale_2020.parquet') 

Working on month: 01
Working on page: 1
Working on page: 2
Working on page: 3
Working on page: 4
Working on page: 5
Working on month: 02
Working on page: 1
Working on page: 2
Working on page: 3
Working on page: 4
Working on page: 5
Working on month: 03
Working on page: 1
Working on page: 2
Working on page: 3
Working on page: 4
Working on page: 5
Working on month: 04
Working on page: 1
Working on page: 2
Working on page: 3
Working on page: 4
Working on page: 5
Working on month: 05
Working on page: 1
Working on page: 2
Working on page: 3
Working on page: 4
Working on page: 5
Working on month: 06
Working on page: 1
Working on page: 2
Working on page: 3
Working on page: 4
Working on page: 5
Working on month: 07
Working on page: 1
Working on page: 2
Working on page: 3
Working on page: 4
Working on page: 5
Working on month: 08
Working on page: 1
Working on page: 2
Working on page: 3
Working on page: 4
Working on page: 5
Working on month: 09
Working on page: 1
Working on page: 2
Working on pa

In [41]:
init_parser = create_parser('https://www.zf.ro/burse-fonduri-mutuale/arhiva/2019/01/')
#21:08 start
news_df = pd.DataFrame()
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
#months = ['04', '05', '06', '07', '08', '09', '10', '11', '12']
for month in months:
    print('Working on month: {}'.format(month))
    init_parser = create_parser('https://www.zf.ro/burse-fonduri-mutuale/arhiva/2019/' + month + '/')
    max_page = 1
    for parsed in init_parser.find_all("li"):
        if 'title' in parsed.find_next('a').attrs:
            if 'pagina' in parsed.find_next('a')['title']:
                if int(parsed.find_next('a')['title'][-1:]) > max_page:
                    max_page = int(parsed.find_next('a')['title'][-1:])
    for i in range(1, max_page+1):
        print('Working on page: {}'.format(i))
        
        parser = create_parser('https://www.zf.ro/burse-fonduri-mutuale/arhiva/2019/'+ month + '/page/' + str(i))
        
        for article_h2 in parser.findAll('h2'):
            href = article_h2.find_next('a', href=True)['href']
            title = article_h2.find_next('a', href=True)\
                                          .contents[0]\
                                          .get_text()\
                                          .strip()
            try:
                parser_article = create_parser('https://www.zf.ro' + article_h2.find_next('a', href=True)['href'])

                json_scr = json.loads(parser_article.find_all('script')[2].text, strict = False)
                date_published = json_scr['datePublished']
                headline = json_scr['headline']
                article_section = json_scr['articleSection']
                description = json_scr['description']
                if 'name' in json_scr['author']:
                    author_name = json_scr['author']['name']
                else:
                    author_name = None
                article_text = json_scr['articleBody']
                rec_refs = parser_article.find_all("a", {"class": "thumb picture-type"})
                rec_hrefs = []
                for ref in range(len(rec_refs)):
                    rec_hrefs.append(rec_refs[ref]['href'])

                dict_append = {'href': href,
                              'title': title,
                              'date_published': date_published,
                              'headline': headline,
                              'article_section': article_section,
                              'description': description,
                              'author_name': author_name,
                              'article_text': article_text,
                              'rec_hrefs': rec_hrefs}
                news_df = news_df.append(dict_append, ignore_index = True)
            except:
                continue
news_df.to_parquet('datasets/df_burse_fonduri_mutuale_2019.parquet') 

Working on month: 01
Working on page: 1
Working on page: 2
Working on page: 3
Working on page: 4
Working on page: 5
Working on month: 02
Working on page: 1
Working on page: 2
Working on page: 3
Working on page: 4
Working on page: 5
Working on month: 03
Working on page: 1
Working on page: 2
Working on page: 3
Working on page: 4
Working on page: 5
Working on month: 04
Working on page: 1
Working on page: 2
Working on page: 3
Working on page: 4
Working on page: 5
Working on month: 05
Working on page: 1
Working on page: 2
Working on page: 3
Working on page: 4
Working on page: 5
Working on month: 06
Working on page: 1
Working on page: 2
Working on page: 3
Working on page: 4
Working on page: 5
Working on month: 07
Working on page: 1
Working on page: 2
Working on page: 3
Working on page: 4
Working on page: 5
Working on month: 08
Working on page: 1
Working on page: 2
Working on page: 3
Working on page: 4
Working on page: 5
Working on month: 09
Working on page: 1
Working on page: 2
Working on pa

In [9]:
test_df = pd.read_parquet('datasets/df_finante_pers_2022.parquet')

In [10]:
test_df

,href,title,date_published,headline,article_section,description,author_name,article_text,rec_hrefs
0,/eveniment/salariul-minim-romania-515-euro-mij...,"Salariul minim din România, de 515 euro, este ...",2022-01-31T18:00+03:00,"Salariul minim din România, de 515 euro, este ...",Eveniment,"Salariul minim din Ro­mâ­nia, la mijlocul clas...",Ramona Cornea,\n ♦&amp;nbsp;Salariul minim din România est...,[/eveniment/cea-importanta-lege-an-legea-buget...
1,/banci-si-asigurari/cat-scump-imprumuta-romani...,Cât de scump se împrumută România având în ved...,2022-01-31T09:00+03:00,Cât de scump se împrumută România având în ved...,Bănci și Asigurări,Nivelul costurilor la care se împrumută Minist...,Claudia Medrega,\nNivelul costurilor la care se împrumută Min...,[]
2,/eveniment/oecd-v-am-invitat-pentru-ca-pib-ul-...,OECD: V-am invitat pentru că PIB-ul pe cap de ...,2022-01-31T00:08+03:00,OECD: V-am invitat pentru că PIB-ul pe cap de ...,Eveniment,"Venit în premieră în România, şeful Organizaţi...",Răzvan Botea,"\nVenit în premieră în România, şeful Organiz...",[/eveniment/romania-a-recuperat-cel-mai-bine-d...
3,/burse-fonduri-mutuale/finante-personale-unde-...,Finanţe personale. De unde până acum cinci ani...,2022-01-31T00:07+03:00,Finanţe personale. De unde până acum cinci ani...,Burse - Fonduri mutuale,Dacă în urmă cu 5-6 ani tranzacţiile anemice ş...,Liviu Popescu,\nDacă în urmă cu 5-6 ani tranzacţiile anemic...,[]
4,/eveniment/cum-au-fost-veniturile-si-cheltuiel...,Cum au fost veniturile şi cheltuielile statulu...,2022-01-31T00:06+03:00,Cum au fost veniturile şi cheltuielile statulu...,Eveniment,Deficitul bugetar pe 2021 a fost de circa 80 d...,Răzvan Botea,\n ♦&amp;nbsp;Cheltuielile bugetului de stat...,[]
...,...,...,...,...,...,...,...,...,...
1001,/burse-fonduri-mutuale/ce-puteti-citi-in-editi...,Ce puteţi citi în ediţia Business Magazin din ...,2022-12-05T07:21+03:00,Ce puteţi citi în ediţia Business Magazin din ...,Burse - Fonduri mutuale,"La 27 de ani, Ştefan Surcel are un portofoliu ...",Tibi Oprea\n,"\n ♦&amp;nbsp; CTP NV, UiPath, Xiaomi, WDP, ...",[/burse-fonduri-mutuale/puteti-citi-editia-bus...
1002,/burse-fonduri-mutuale/s-au-activat-marii-inve...,S-au activat marii investitori la FP: 3% din a...,2022-12-05T07:00+03:00,S-au activat marii investitori la FP: 3% din a...,Burse - Fonduri mutuale,"Vineri, la ora 15:00, în sistemul Bursei de Va...",Tibi Oprea\n,\n ♦&amp;nbsp; Aceasta este cea mai mare tra...,"[http://www.zfcorporate.ro/rd-21353400, /burse..."
1003,/burse-fonduri-mutuale/cele-sapte-fonduri-pens...,Cele şapte fonduri de pensii private Pilon II ...,2022-12-04T15:11+03:00,Cele şapte fonduri de pensii private Pilon II ...,Burse - Fonduri mutuale,"Din vara lui 2019 şi până în octombrie 2022, c...",Liviu Popescu,"\n ♦ Cu tot cu statul român, 8 milioane de v...",[/burse-fonduri-mutuale/zf-piata-capital-2022-...
1004,/burse-fonduri-mutuale/companiile-din-prima-li...,Companiile din prima ligă bursieră au pierdut...,2022-12-02T00:07+03:00,Companiile din prima ligă bursieră au pierdut...,Burse - Fonduri mutuale,Capitalizarea cumulată a companiilor din com­p...,Eduard Ivanovici,\nCapitalizarea cumulată a companiilor din co...,[/burse-fonduri-mutuale/trei-ani-fondurile-pen...
